# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
# https://github.com/prateekralhan/Gesture-Recognition-Case-study-IIITB-Assignment-

In [2]:
import numpy as np
import os
# from scipy.misc import imread, imresize
from skimage.io import imread
from skimage.transform import resize as imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow.keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 32#experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [9]:
img_idx =  [5,7,11,13,14,16,18,19,20,21,22,23,24,25,26,27,28]#create a list of image numbers you want to use for a particular video
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    
    y = 100
    z = 100
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = image.resize(100, 100)
                    
                    batch_data[folder,idx,:,:,0] /= 255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] /= 255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if len(folder_list)%batch_size != 0:
            # updated the batch size and yield
            batch_size = len(folder_list)%batch_size
            num_batches = len(source_path)//batch_size# calculate the number of batches
            for batch in range(num_batches): # we iterate over the number of batches
                batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                        
                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes
                        image = image.resize(100, 100)
                        
                        batch_data[folder,idx,:,:,0] /= 255#normalise and feed in the image
                        batch_data[folder,idx,:,:,1] /= 255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] /= 255 #normalise and feed in the image
                        
                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [12]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here
inputShape = (len(img_idx), 100, 100, 3)

model = Sequential([
    Conv3D(16, (5, 5, 5), activation='relu', input_shape=inputShape),
    MaxPooling3D((2,2,2), padding='same'),
    BatchNormalization(),

    Conv3D(32, (3, 3, 3), activation='relu'),
    MaxPooling3D(pool_size=(2, 2, 2), padding='same'),
    BatchNormalization(),

    Conv3D(64, (3, 3, 3), activation='relu'),
    MaxPooling3D(pool_size=(2, 2, 2), padding='same'),
    BatchNormalization(),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),

    Dense(5, activation='softmax')
])

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [14]:
optimiser = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 13, 96, 96, 16)    6016      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 48, 48, 16)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 48, 48, 16)     64        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 5, 46, 46, 32)     13856     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 23, 23, 32)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 23, 23, 32)     128       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 21, 21, 64)     5

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [16]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,
  verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,
  min_lr = 0)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [18]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

c:\Users\U6083443\Documents\Repositories\Personal\Upgrad-PGD-ML---AI\env\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Source path =  ./Project_data/train ; batch size = 32
Epoch 1/10
21/21 [==============================] - 135s 6s/step - loss: 1.6085 - categorical_accuracy: 0.2106 - val_loss: 1.6119 - val_categorical_accuracy: 0.1250

Epoch 00001: saving model to model_init_2021-05-2319_32_53.423896\model-00001-1.60772-0.21053-1.61193-0.12500.h5
Epoch 2/10
21/21 [==============================] - 136s 6s/step - loss: 1.6048 - categorical_accuracy: 0.2024 - val_loss: 1.6068 - val_categorical_accuracy: 0.0625

Epoch 00002: saving model to model_init_2021-05-2319_32_53.423896\model-00002-1.60966-0.18768-1.60681-0.06250.h5
Epoch 3/10
21/21 [==============================] - 147s 7s/step - loss: 1.6069 - categorical_accuracy: 0.2223 - val_loss: 1.6299 - val_categorical_accuracy: 0.1875

Epoch 00003: saving model to model_init_2021-05-2319_32_53.423896\model-00003-1.60648-0.22409-1.62990-0.18750.h5
Epoch 4/10
21/21 [==============================] - 143s 7s/step - loss: 1.6029 - categorical_accuracy: 0.244

In [ ]:
# https://github.com/santhoshpkumar/hand-gesture-recognition-using-neural-networks

# https://www.kaggle.com/imsparsh/gesture-recognition-conv3d-conv2d-rnn

# https://github.com/prateekralhan/Gesture-Recognition-Case-study-IIITB-Assignment-/blob/master/Gesture_Recognition_Case-Study_final.ipynb

# https://github.com/Smitan94/Data-Science/blob/master/Gesture%20recognition%20-%20Neural%20Network.ipynb

